# Testing DMD against rational adversaries

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import sys
sys.path.append('../')

In [3]:
import source.players as pl
import source.parsers as parsers
import source.game as game
import source.player as player
import source.environment as environment
import source.errors as errors
import source.runner as runner
import source.util as util
import source.players.attackers as attackers
import source.players.base_defenders as base_defenders
import source.players.defenders as defenders
import source.players.holmes as holmes
import source.players.sad as sad
import source.players.b2bw2w as b2bw2w
import source.players.belief_max as bm
import source.players.dmd as dmd

import concurrent.futures
import pkgutil
import logging
import sys
from source.util import *
import numpy as np
import pandas as pd
from importlib import *
import time
import os
import re
from copy import deepcopy, copy
import datetime
from contextlib import redirect_stdout
from random import shuffle

In [4]:
# run once

lvl = logging.DEBUG
logger = logging.getLogger("source")
logger.setLevel(lvl)
#ch = logging.StreamHandler(sys.stdout)
ch = logging.FileHandler("status", mode='w')
ch.setLevel(lvl)
#formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
formatter = logging.Formatter('%(message)s')
ch.setFormatter(formatter)
logger.addHandler(ch)

In [5]:
lvl = logging.WARNING
logger.setLevel(lvl)
ch.setLevel(lvl)

In [6]:
# if you change something in the files, don't forget to reload
reload(player)
reload(base_defenders)
reload(attackers)
reload(defenders)
reload(errors)
reload(environment)
reload(runner)
reload(parsers)
reload(game)
reload(holmes)
reload(sad)
reload(b2bw2w)
reload(util)
reload(bm)
reload(dmd)

<module 'source.players.dmd' from '..\\source\\players\\dmd.py'>

### Creating configurations to save into a csv file

In [7]:
time_horizon = 100
T = 6
#targets = util.gen_targets(T, low=1, high=10)
targets = util.gen_tar_with_len(T)
values = tuple((v, v) for v in targets)
print(values)
#obs = gen_observabilities_correlated_with_values(values)
#obs = {0: 0.75, 1: 0.75, 2: 0.75, 3: 0.75, 4: 0.75}
#obs_vect = [obs.get(i) for i in range(len(targets))]
#generating pure strat and some rand mixed strat stochastic attackers
distributions = []
for i in range(10):
    distributions.append(tuple(util.gen_distr(T)))
print(distributions)
chosen_distr = tuple(util.gen_distr(T))
print("chosen = ", chosen_distr)

((0.699, 0.699), (0.977, 0.977), (0.98, 0.98), (0.771, 0.771), (0.914, 0.914), (0.692, 0.692))
[(0.095, 0.544, 0.102, 0.167, 0.056, 0.036), (0.14014014014014015, 0.1111111111111111, 0.3493493493493493, 0.29729729729729726, 0.04404404404404404, 0.058058058058058054), (0.05094905094905095, 0.2107892107892108, 0.4615384615384616, 0.17582417582417584, 0.05694305694305695, 0.04395604395604396), (0.079, 0.225, 0.04, 0.395, 0.157, 0.104), (0.096, 0.043, 0.303, 0.325, 0.224, 0.009), (0.012, 0.052, 0.103, 0.436, 0.22, 0.177), (0.06293706293706293, 0.3766233766233766, 0.03196803196803197, 0.23476523476523473, 0.17982017982017978, 0.11388611388611387), (0.10210210210210209, 0.13613613613613615, 0.07807807807807808, 0.6126126126126126, 0.03203203203203203, 0.03903903903903904), (0.286, 0.114, 0.431, 0.055, 0.052, 0.062), (0.02897102897102897, 0.5124875124875125, 0.015984015984015984, 0.0799200799200799, 0.03196803196803197, 0.33066933066933063)]
chosen =  (0.07292707292707293, 0.29570429570429574,

In [20]:
g = game.Game(values, time_horizon)
g.attackers = [1]
g.defenders = [0]
d = [dmd.DirichletMultinomialDefender(g, 0)]
prof = []
for distr in distributions:
    prof.append(attackers.StochasticAttacker(g, 1, 1, *distr))
for i in range(10):
    prof.append(attackers.SUQR(g,1))
chosen_a_prof = (attackers.StackelbergAttacker(g, 1))
a = [deepcopy(chosen_a_prof)]
prof.append(chosen_a_prof)
shuffle(prof)
print("Real attacker is: ", chosen_a_prof)
g.set_players(d, a, prof)

Real attacker is:  sta1


In [21]:
def print_header(targets, profiles):
    return "Name,T," + ",".join(str(i) for i in range(len(targets))) + ",Defender,Attacker," + ",".join(["Profile" for x in range(len(profiles))]) + "\n" 

In [22]:
def print_row(targets, time_horizon, d, a, p):
    return ",".join([str(i) for i in ([d+"_vs_"+a] + [time_horizon] + targets + [d] + [a] + [x for x in p]) ])+"\n"

In [23]:
(print_header(targets, prof))

'Name,T,0,1,2,3,4,5,Defender,Attacker,Profile,Profile,Profile,Profile,Profile,Profile,Profile,Profile,Profile,Profile,Profile,Profile,Profile,Profile,Profile,Profile,Profile,Profile,Profile,Profile,Profile\n'

In [24]:
defender =["DMD1"]

In [25]:
profile_strings = []
for p in prof:
    profile_strings.append(util.print_adv(p))

In [7]:
experiments_folder = "../NewExperiments"
exp_subfolder = "/vsSUQR"
batch_name = "dmd"
batch_file = batch_name + ".csv"
batch_path = experiments_folder + exp_subfolder + "/" + batch_file

### Generating the csv and checking everything is fine

In [27]:
with open(batch_path, "w+") as f:
    f.write(print_header(targets, prof))
    f.write(print_row(targets, time_horizon,defender[0],util.print_adv(a[0]),profile_strings))

In [28]:
pd.read_csv(batch_path)

,Name,T,0,1,2,3,4,5,Defender,Attacker,...,Profile.11,Profile.12,Profile.13,Profile.14,Profile.15,Profile.16,Profile.17,Profile.18,Profile.19,Profile.20
0,DMD1_vs_sta1,100,0.772,0.914,0.94,0.758,0.728,0.858,DMD1,sta1,...,suqr1-8.299-0.073,sto1-0.024024024024024024-0.12812812812812813-...,suqr1-13.642-0.54,suqr1-8.918-0.982,suqr1-12.686-0.222,suqr1-7.565-0.457,suqr1-10.585-0.262,suqr1-12.485-0.815,suqr1-7.811-0.677,sto1-0.07607607607607607-0.47147147147147145-0...


### Parsing the csv to generate a batch containing the configurations and running them

In [8]:
#os.mkdir("../NewExperiments/Exp2/belief_max_obs_vs_no_obs")

In [9]:
b = runner.Batch(batch_path, experiments_folder + exp_subfolder)

In [10]:
b.parse_batch()

In [11]:
batches = []
batches.append(b)

In [17]:
with concurrent.futures.ProcessPoolExecutor(None) as executor:
    futures = {}
    logger.info(str(datetime.datetime.now()))
    for b in batches:
        b.run(futures, executor, n=2)

### Collecting results of the runs

In [18]:
with open('progress', 'w') as f:
    with redirect_stdout(f):
        for b in batches:
            b.collect(futures)
            confs = b.configurations
            print(b.results_folder_path)
            dlist = [{"name": c.game.players[0].__class__.name,
                      "avgs": c.stats["exp_regret"], 
                      "lb": c.stats["lb_exp_regret"],
                      "ub": c.stats["ub_exp_regret"]}
                     for c in confs]
            util.plot_dicts(dlist, save=True, show=False, semilog=True,
                            path=b.results_folder_path,
                            title=confs[0].game.players[1].__class__.name)

### Checking everything was fine by running the two configurations "step-by-step"

In [12]:
gm = deepcopy(batches[0].configurations[0].game)

In [13]:
gm

<Game values:[[0.772, 0.772], [0.914, 0.914], [0.94, 0.94], [0.758, 0.758], [0.728, 0.728], [0.858, 0.858]] players{1: <SUQR id:1 resources:1>, 0: <DirichletMultinomialDefender id:0 resources:1>} time_horizon:100>

In [14]:
e = runner.Experiment(gm)

In [15]:
print(gm.players[0].profiles[0])

sto1-0.002002002002002002-0.028028028028028024-0.10010010010010009-0.07307307307307306-0.38538538538538536-0.41141141141141135


In [16]:
for i in range(50):
    e.run_interaction()
    print(e.game.strategy_history[-1])
    #print(util.game_str(e.game, lenght=7))
    print(e.game.history[-1])
    #print(list(e.agent.belief.pr.items())[2])

{0: [0, 0, 1, 0, 0, 0], 1: [0.19622380639243953, 0.211862160553873, 2.555674674120298e-07, 0.1947459477604799, 0.19161648053245603, 0.2055513491932841]}
{1: [0], 0: [2]}
{0: (0, 0, 0, 0, 1, 0), 1: [0.19176691761645362, 0.20705007326034536, 0.2099775591508044, 0.1903226260203719, 2.2274580241250015e-07, 0.20088260120622228]}
{1: [3], 0: [4]}
{0: (0, 0, 0, 0, 1, 0), 1: [0.19176691761645362, 0.20705007326034536, 0.2099775591508044, 0.1903226260203719, 2.2274580241250015e-07, 0.20088260120622228]}
{1: [1], 0: [4]}
{0: (0, 1, 0, 0, 0, 0), 1: [0.19563776943499372, 2.5125174187760156e-07, 0.2142159962430517, 0.1941643245374944, 0.19104420369555108, 0.20493745483716713]}
{1: [0], 0: [1]}
{0: (0, 1, 0, 0, 0, 0), 1: [0.19563776943499372, 2.5125174187760156e-07, 0.2142159962430517, 0.1941643245374944, 0.19104420369555108, 0.20493745483716713]}
{1: [2], 0: [1]}
{0: (0, 1, 0, 0, 0, 0), 1: [0.19563776943499372, 2.5125174187760156e-07, 0.2142159962430517, 0.1941643245374944, 0.19104420369555108, 0.20

In [19]:
e.game

<Game values:[[0.902, 0.902], [0.9790000000000001, 0.9790000000000001], [0.848, 0.848], [0.976, 0.976], [0.956, 0.956], [0.912, 0.912]] players{1: <StochasticAttacker id:1 resources:1>, 0: <DirichletMultinomialDefender id:0 resources:1>} time_horizon:100>

In [28]:
e.agent.belief.loglk

{<StochasticAttacker id:1 resources:1>: None,
 <StochasticAttacker id:1 resources:1>: None,
 <StochasticAttacker id:1 resources:1>: -3.6848977664046223,
 <StochasticAttacker id:1 resources:1>: -1.6094379124340925,
 <StochasticAttacker id:1 resources:1>: None,
 <StochasticAttacker id:1 resources:1>: -3.688879454114018,
 <StochasticAttacker id:1 resources:1>: None,
 <StochasticAttacker id:1 resources:1>: None,
 <StochasticAttacker id:1 resources:1>: None,
 <StochasticAttacker id:1 resources:1>: -3.688879454114018,
 <StochasticAttacker id:1 resources:1>: None,
 <StochasticAttacker id:1 resources:1>: -3.190303301163861,
 <StochasticAttacker id:1 resources:1>: -1.2740214327993638,
 <StochasticAttacker id:1 resources:1>: -0.09126790561131946,
 <StochasticAttacker id:1 resources:1>: -3.6848977664046223,
 <StochasticAttacker id:1 resources:1>: -0.1133238910766163,
 <StochasticAttacker id:1 resources:1>: -3.32124264636016,
 <StochasticAttacker id:1 resources:1>: -1.6956695963863746,
 <Stochasti

In [29]:
gm1 = deepcopy(batches[0].configurations[1].game)

In [30]:
gm1

<GameWithObservabilities values:[[5.0, 5.0], [3.0, 3.0], [8.0, 8.0], [4.0, 4.0], [9.0, 9.0]] players{1: <StackelbergAttacker id:1 resources:1>, 0: <FB id:0 resources:1>} time_horizon:1000>

In [31]:
e1 = runner.Experiment(gm1)

In [32]:
for i in range(900):
    e1.run_interaction_with_observations()
    print(e1.game.strategy_history[-1])
    #print(util.game_str(e.game, lenght=7))
    print(e1.game.history[-1])
    print(e1.game.observation_history[-1])
    #print(list(e.agent.belief.pr.items())[2])

{0: (0, 0, 0, 1, 0), 1: [0, 0, 0, 0, 1]}
{1: [4], 0: [3]}
{0: 1, 1: 1, 2: 1, 3: 0, 4: 1}
{0: (0, 0, 0, 0, 1), 1: [0, 0, 1, 0, 0]}
{1: [2], 0: [4]}
{0: 1, 1: 1, 2: 1, 3: 1, 4: 0}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [2]}
{0: 1, 1: 1, 2: 0, 3: 0, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [0]}
{0: 1, 1: 1, 2: 1, 3: 1, 4: 0}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [2]}
{0: 1, 1: 1, 2: 1, 3: 1, 4: 0}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 1, 2: 1, 3: 1, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [0]}
{0: 1, 1: 1, 2: 1, 3: 0, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}


{0: 1, 1: 0, 2: 1, 3: 1, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 1, 2: 1, 3: 0, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 0, 1: 1, 2: 1, 3: 1, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [2]}
{0: 1, 1: 1, 2: 1, 3: 1, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [0]}
{0: 1, 1: 1, 2: 1, 3: 1, 4: 0}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 0, 1: 1, 2: 1, 3: 1, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [2]}
{0: 1, 1: 1, 2: 1, 3: 0, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1

{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [0]}
{0: 1, 1: 1, 2: 1, 3: 1, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [2]}
{0: 1, 1: 0, 2: 0, 3: 1, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 1, 2: 1, 3: 1, 4: 0}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [2]}
{0: 1, 1: 1, 2: 1, 3: 1, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [0]}
{0: 0, 1: 1, 2: 1, 3: 1, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 0, 2: 1, 3: 1, 4: 0}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [2]}
{0: 1, 1: 0, 2: 

{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [2]}
{0: 1, 1: 1, 2: 1, 3: 1, 4: 0}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [2]}
{0: 1, 1: 1, 2: 1, 3: 0, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 0, 2: 0, 3: 0, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 1, 2: 0, 3: 1, 4: 0}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [2]}
{0: 0, 1: 1, 2: 1, 3: 1, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 0, 1: 1, 2: 1, 3: 0, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [2]}
{0: 1, 1: 1, 2: 

{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [2]}
{0: 0, 1: 1, 2: 1, 3: 1, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 1, 2: 1, 3: 1, 4: 0}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 1, 2: 1, 3: 1, 4: 0}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 1, 2: 0, 3: 0, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 0, 1: 0, 2: 1, 3: 1, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 1, 2: 1, 3: 1, 4: 0}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 0, 1: 1, 2: 

{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 0, 2: 1, 3: 0, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 1, 2: 1, 3: 1, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 0, 1: 1, 2: 0, 3: 0, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 0, 2: 1, 3: 1, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 1, 2: 1, 3: 1, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [0]}
{0: 1, 1: 0, 2: 1, 3: 0, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 1, 2: 

{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 0, 2: 0, 3: 1, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 1, 2: 1, 3: 1, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [2]}
{0: 0, 1: 1, 2: 0, 3: 1, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 0, 2: 0, 3: 1, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 0, 2: 1, 3: 1, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 1, 2: 1, 3: 1, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 1, 2: 

{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [2]}
{0: 1, 1: 1, 2: 0, 3: 1, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [2]}
{0: 1, 1: 1, 2: 0, 3: 0, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 1, 2: 1, 3: 1, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 1, 2: 1, 3: 0, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 1, 2: 1, 3: 1, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 1, 2: 1, 3: 1, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 1, 2: 

{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 1, 2: 1, 3: 1, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 0, 1: 1, 2: 0, 3: 1, 4: 0}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 1, 2: 0, 3: 1, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 1, 2: 0, 3: 1, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [2]}
{0: 1, 1: 0, 2: 1, 3: 1, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [2]}
{0: 1, 1: 0, 2: 1, 3: 1, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 1, 2: 

{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 0, 2: 1, 3: 0, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [2]}
{0: 1, 1: 1, 2: 1, 3: 0, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 1, 2: 1, 3: 1, 4: 0}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 0, 2: 1, 3: 1, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [2]}
{0: 1, 1: 0, 2: 0, 3: 1, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [0]}
{0: 1, 1: 0, 2: 1, 3: 1, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 0, 2: 